# extract-pages-from-mongo
SanjayKAroraPhD@gmail.com <br>
November 2018

## Description
This notebook extracts groups of pages from mongodb by firm_name to create firm-centric page output files that can later be topic modeled.  In doing so, it removes repetitive content (e.g., repeated menu items) and garbage content (e.g., improperly parsed HTML code). 

## Change log
v2 fixes some bugs.  

## TODO:
* Why are there so few firms with home pages being output (857?)
* Need to make better use of all pages in the site, e.g., to improve data quality and use additional paragraph data found on non-homepages 

In [2]:
# import data processing and other libraries
import csv
import sys
import requests
import os
import re
import pprint
import pymongo
import traceback
from time import sleep
import requests
import pandas as pd
import io
from IPython.display import display
import time
import numpy as np
from bs4 import BeautifulSoup
import string

In [3]:
MONGODB_DB = "FirmDB_20181116"
MONGODB_COLLECTION = "pages_COMBINED"
CONNECTION_STRING = "mongodb://localhost"

client = pymongo.MongoClient(CONNECTION_STRING)
db = client[MONGODB_DB]
col = db[MONGODB_COLLECTION]

DATA_DIR = '/Users/sarora/dev/EAGER/data/orgs/depth0_pages/'

In [4]:
# gather unique firm_names from mongodb

def get_firm_aggregates ():
    query = [ { "$group": {"_id":"$firm_name" , "number":{"$sum":1}} } ]
    results = col.aggregate(query)

    mongo_dict = {}
    for result in results:
        key = (result['_id'])
        if key:
            mongo_dict[key[0]] = result['number']
        else:
            mongo_dict['NA'] = result['number']
    
    return mongo_dict

results_dict = get_firm_aggregates()
firm_names = results_dict.keys()
pp = pprint.PrettyPrinter()
pp.pprint(firm_names)

[u'Little Kids',
 u'Daylight Solutions',
 u'Biocon Limited',
 u'Sony Corporation',
 u'Magna International Inc.',
 u'ENI Technology',
 u'Honeywell International Inc.',
 u'Aurrion',
 u'The Jackson Laboratory',
 u'Hewlett Packard Enterprise Development LP',
 u'CUMMINS FILTRATION IP',
 u'Custom Electronics Inc.',
 u'H R D CORPORATION',
 u'GENERAL MOTORS LLC',
 u'FLIR Systems',
 u'Sola U.S.A. Inc.',
 u'Chromalox',
 u'Empire Technology Development LLC',
 u'Alcon Research',
 u'Wyatt Technology Corporation',
 u'Southwest Research Institute',
 u'Tokyo Ohka Kogyo Co.',
 u'Samsung Electronics',
 u'Fairchild Semiconductor Corporation',
 u'Supremex Inc.',
 u'Calient Technologies',
 u'Infineon Technologies Americas Corp.',
 u'Ideal Power Inc.',
 u'NuOrtho Surgical',
 u'Easel Biotechnologies',
 u'L-3 Communications Cincinnati Electronics Corporation',
 u'SunEdison Semiconductor Limited',
 u'IDEA TREE',
 u'Solarmer Energy',
 u'Braun Intertec Geothermal',
 u'Fuji Electric Co.',
 u'The Babcock & Wilcox 

 u'TRI ALPHA ENERGY',
 u'Renmatix',
 u'Zygo Corporation',
 u'PACCAR Inc',
 u'Evernote Corporation',
 u'Entegris',
 u'IDEX Health & Science LLC',
 u'Echogen Power Systems',
 u'Lion Copolymer Geismar',
 u'Nanoridge Materials',
 u'Rebellion Photonics',
 u'Big Belly Solar',
 u'Nokomis',
 u'SUMITOMO WIRING SYSTEMS',
 u'Zoetis Services LLC',
 u'ExxonMobil Research and Engineering Company',
 u'VINDICO NANOBIO TECHNOLOGY INC.',
 u'HNO Greenfuels',
 u'Esolar',
 u'Bayer Cropscience AG',
 u'EMD Technologies Inc.',
 u'Newlans',
 u'HIQ SOLAR',
 u'Nederlandse Organisatie voor toegepast-natuurwetenschappelijk onderzoek TNO',
 u'U S MICROPOWER INC',
 u'Hewlett-Packard Development Company',
 u'Arkema Inc.',
 u'Floadia Corporation',
 u'Entech Solar',
 u'Global Solar Water Power Systems',
 u'PortaFire',
 u'Ormat Technologies Inc.',
 u'Calysta',
 u'MCI',
 u'Singular Bio',
 u'Xyleco',
 u'Bigelow Aerospace',
 u'Pacific Light Technologies',
 u'Tesla Nanocoatings',
 u'Integrated Solar Technology',
 u'FULL CIR

In [14]:
# remove html content
def is_javascript (x):
    match_string = "(CDATA|return\s+true|return\s+false|function|\w+\(.*?\);|\w{2,}[\\.|:]+\w{2,}|'\w+':\s+'\w+')|{|}|\r|\n|\/\/"
    # capture CDATA; function declarations; function calls; word sequences separated by a period (e.g., denoting paths)
    regex = re.findall(match_string, x) 
    # check to see if the regex finds some percentage of the words look like javascript patterns
    if (len(regex) / float(len(x.split())) > .10) and len(regex) > 3:
        return True 
    else:
        return False

def clean_page_content (text_list):
    # remove whatever we think is html
    removed_html = filter(lambda x: not( bool(BeautifulSoup(x, "html.parser").find()) ), text_list)
    # remove content that looks like javascript 
    removed_js = filter(lambda x: not (is_javascript(x)), removed_html)
    # add other checks here as needed

    return removed_js
    

# iterate through each firm, get all pages associated with a firm, and produce data structure
# url --> depth
#     --> content (list)
# return data structure
def process_firm (firm_name): 
    regex = '^' + re.escape(firm_name) + '$'
    results = col.find( {"$and":[ {"firm_name": re.compile(regex, re.IGNORECASE) }, {"depth":0}]} )
    
    firm_pages_dict = {}
    depth0_page_text = [] # home page
    for result in results:
        key = (result['url'])
        if key:
            page_dict = {}
            depth = result['depth'][0]
            page_dict['depth'] = depth
            page_dict['domain'] = result['domain'][0]
            page_dict['firm_name'] = firm_name
            clnd_text = clean_page_content(result['full_text'])
            page_dict['clnd_text'] = clnd_text
            firm_pages_dict[key[0]] = page_dict
            
            if depth == 0:
                depth0_page_text = clnd_text
        else:
            continue
            
    return firm_pages_dict, depth0_page_text
# TODO: identify which pieces of content are common across all sites, and remove those
# def clean_content(firm_dict): 

In [22]:
# regex test 
regex = re.findall(r"(CDATA|return\s+true|return\s+false|function|\w+\(.*?\);|\w{2,}[\\.|:]+\w{2,}|'\w+':\s+'\w+|\\')", 
                   "CDATA function contact-us javascript.function linker:autoLink www.littlekidsinc.com fxnCall(param.param); email@dextr.us 'type': 'image' return true return false rev7bynlh\\u00252bvcgrjg\\") # last part is words sequences separated by punct
print (regex)

['CDATA', 'function', 'javascript.function', 'linker:autoLink', 'www.littlekidsinc', 'fxnCall(param.param);', 'dextr.us', "'type': 'image", 'return true', 'return false', 'rev7bynlh\\u00252bvcgrjg']


In [15]:
firm_pages_dict, depth0_page_text = process_firm ('Brother International Corporation')
print (depth0_page_text)

[u'For Home', u'For Business', u'For Home', u'For Business', u'Login', u' Products', u'Products ', u'Products', u'For Home', u'Products', u'For Business', u'U.S.A. | Global Network', u'\xa9 2018 Brother International Corporation ', u'Global Site']


In [16]:
# run
pp = pprint.PrettyPrinter()
for firm_name in firm_names: 
    print ("Working on " + firm_name)
    firm_pages_dict, depth0_page_text = process_firm (firm_name)
    # pp.pprint(depth0_page_text)
    if depth0_page_text: 
        file = re.sub('\.|\/', '_', firm_name) + '.txt'
        with io.open(DATA_DIR + file,'w',encoding='utf8') as f:
            f.write ('\n'.join (depth0_page_text))

Working on Little Kids
Working on Daylight Solutions
Working on Biocon Limited
Working on Sony Corporation
Working on Magna International Inc.
Working on ENI Technology
Working on Honeywell International Inc.
Working on Aurrion
Working on The Jackson Laboratory
Working on Hewlett Packard Enterprise Development LP
Working on CUMMINS FILTRATION IP
Working on Custom Electronics Inc.
Working on H R D CORPORATION
Working on GENERAL MOTORS LLC
Working on FLIR Systems
Working on Sola U.S.A. Inc.
Working on Chromalox
Working on Empire Technology Development LLC
Working on Alcon Research
Working on Wyatt Technology Corporation
Working on Southwest Research Institute
Working on Tokyo Ohka Kogyo Co.
Working on Samsung Electronics
Working on Fairchild Semiconductor Corporation


/Users/sarora/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:219: UserWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/Users/sarora/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://www.hp.com/go/samsung" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Working on Supremex Inc.
Working on Calient Technologies
Working on Infineon Technologies Americas Corp.
Working on Ideal Power Inc.
Working on NuOrtho Surgical
Working on Easel Biotechnologies
Working on L-3 Communications Cincinnati Electronics Corporation
Working on SunEdison Semiconductor Limited
Working on IDEA TREE
Working on Solarmer Energy
Working on Braun Intertec Geothermal
Working on Fuji Electric Co.
Working on The Babcock & Wilcox Company
Working on Rima Enterprises
Working on Saint-Gobain Adfors Canada
Working on Agilent Technologies
Working on Toray Industries Inc. 
Working on Christie Digital Systems
Working on SYNTHOMER USA LLC
Working on Nissan North America


/Users/sarora/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:219: UserWarning: "//" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


Working on Shin-Etsu Chemical Co.
Working on FUJIFILM Dimatix
Working on SUNSALUTER
Working on SII Semiconductor Corporation
Working on Prosoft International
Working on Energen
Working on KR Design House
Working on Akron Polymer Systems
Working on Revivicor
Working on Caribou Biosciences
Working on Syngenta Participations AG
Working on BERKEN ENERGY LLC
Working on ADASA INC.
Working on Abbott Molecular Inc.
Working on PLEX LLC
Working on Carver Scientific
Working on E. Tech Incorporation
Working on Canon
Working on Renewable Power Conversion
Working on Thermo Fisher Scientific Inc.
Working on Toray Plastics (America)
Working on Algenol Biotech LLC
Working on Cadence Design Systems
Working on ACACIA RESEARCH GROUP LLC
Working on Cabot Corporation
Working on Genesco Inc.
Working on CARBO Ceramics Inc.
Working on Metabolix
Working on TEKNOR APEX COMPANY
Working on One Earth Designs Inc.
Working on Commonwealth Scientific & Industrial Research Organization
Working on Siemens Industry
Worki

...

Working on Strategic Solar Energy
Working on GlaxoSmithKline Biologicals
Working on iBio
Working on Voxtel
Working on ENI S.p.A.
Working on Sigma-Aldrich Co. LLC
Working on Bridgestone Corporation
Working on Turtle Beach Corporation
Working on Palo Alto Research Center Incorporated
Working on Scientific Design Company
Working on SAE Magnetics (HK) Ltd.
Working on JFE STEEL CORPORATION
Working on Silicon Genesis Corporation
Working on JAC Products Inc.
Working on Vaxiion Therapeutics
Working on Johns Manville
Working on Watlow Electric Manufacturing Company
Working on Rockwell Collins
Working on Transposagen Biopharmaceuticals
Working on Conexant Systems
Working on Wave Energy Conversion Corporation of America
Working on Hadasit Medical Research Services & Development Company Ltd.
Working on Nippon Chemi-Con Corporation
Working on STMicroelectronics Limited
Working on Coriant Advanced Technology
Working on Dresser-Rand Company
Working on Bluestar Silicones France
Working on Finisar Corp

/Users/sarora/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "https://www.bayer.com" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Working on Nanomix
Working on InVisage Technologies
Working on Atonometrics
Working on ARBOR THERAPEUTICS
Working on Carl Zeiss Microscopy GmbH
Working on Applied Biosystems
Working on THE BOARD INSTITUTE
Working on Relypsa
Working on Ticona LLC
Working on Kao Corporation
Working on Xintec Inc.
Working on MAHLE International GmbH
Working on POLAR LIGHT TECHNOLOGIES AB
Working on Epizyme
Working on Sanken Electric Co.
Working on Alexion Pharmaceuticals
Working on Yissum
Working on WestPoint Home
Working on Georgia-Pacific Gypsum LLC
Working on Korea Kumho Petrochemical Co.
Working on Richtek Technology Corporation
Working on Avery Dennison Corporation
Working on Gtech Corporation


/Users/sarora/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://yissum.co.il/home" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Working on AccuRay Corporation
Working on Gram Power
Working on Teradata US
Working on Rockwell Automation Technologies
Working on NanoOncology
Working on OFS Fitel
Working on ZIH Corp.
Working on DRS Network & Imaging Systems
Working on Travis Industries
Working on Lumentum Operations LLC
